# 从视频到代码：使用 ERNIE 4.5-VL 构建自动化代码生成工作流

### 1. 项目概述

本教程详细介绍并实现一个端到端的工作流，用于将视频格式的编程教程内容，自动化地转换为可执行的前端网页代码。其核心流程是利用多模态大模型对视频内容进行视觉层面的理解，并基于理解生成结构化的代码。

该工作流包含以下关键技术步骤：
1.  **视频获取**: 从指定的URL下载视频源文件。
2.  **视觉信息提取**: 对视频文件进行预处理，按固定时间间隔抽帧，并将图像数据编码。
3.  **分段内容理解**: 将提取的帧序列分块，并行调用ERNIE 4.5-VL模型接口，获取每个视频片段内容的结构化文本描述。
4.  **聚合与代码生成**: 整合所有片段的描述性文本，形成对视频整体内容的全面摘要，并以此为上下文，调用ERNIE 4.5-VL生成最终的HTML代码。
5.  **结果展示**: 将生成的代码保存为HTML文件，并自动在本地浏览器中打开以供验证。

### 2. 核心技术背景：ERNIE 4.5-VL

此工作流的核心是ERNIE 4.5视觉语言模型（Vision-Language Model, VLM），其具备以下与本任务高度相关的特性：

*   **多模态异构MoE架构**: 模型在预训练阶段联合处理文本与视觉两种模态的数据。这种设计使其能够有效捕捉跨模态信息，对于准确理解视频画面中呈现的代码文本和对应的视觉样式至关重要。
*   **视觉理解能力**: ERNIE 4.5-VL在视觉感知、文档与图表理解等基准测试中表现出色。该能力是模型从视频帧中准确识别代码语法、结构和视觉布局的基础。
*   **开源协议**: ERNIE 4.5模型家族基于Apache 2.0协议开源，为技术研究和应用开发提供了便利。

本教程利用ERNIE 4.5-VL的视觉分析和代码生成能力，来完成从非结构化视频到结构化代码的转换任务。
![ERNIE 4.5-VL Performance Comparison](ernie_4.5_vl_performance.png)

### 3. 环境与依赖安装

执行此工作流需要安装用于视频下载、图像处理及API交互的Python库。

运行以下单元格以安装所有必需的依赖项。

In [1]:
# 运行此单元格以安装所有必需的库
!pip install yt-dlp opencv-python openai tqdm

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/


### 4. 库导入与参数配置

此部分导入项目所需的Python模块，并设置全局参数，包括API凭证和目标视频URL。

**说明：**
-   请将 `API_KEY` 变量的值替换为您的百度AI Studio Access Token。
-   `VIDEO_URL` 已预先设置了一个示例视频链接，您可以将其替换为其他前端编程教程的视频链接进行测试。

In [ ]:
import os
import cv2
import time
import base64
import yt_dlp
import shutil
import webbrowser
from openai import OpenAI
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# --- 配置中心 ---

# 1. API凭证配置
# 请替换为您自己的 ERNIE API KEY
API_KEY = "YOUR_API_KEY"
BASE_URL = "https://aistudio.baidu.com/llm/lmapi/v3"
MODEL_NAME = "ernie-4.5-turbo-vl"

# 2. 任务参数配置
VIDEO_URL = "https://www.bilibili.com/video/BV1xQ4y167xr/" # <--- 在此指定目标视频URL
MAX_CONCURRENT_REQUESTS = 4  # 并发API请求数

### 5. 功能函数定义

工作流的各个环节被封装为独立的函数，以实现模块化和代码清晰性。

- **`download_video`**: 负责从指定URL下载视频文件。
- **`extract_frames`**: 负责读取视频文件，提取并编码图像帧序列。
- **`process_chunk_with_retry`**: 负责调用模型API处理单个视频片段的帧序列，并包含网络请求的重试逻辑。
- **`aggregate_and_generate_webpage`**: 负责整合所有片段的分析结果，并调用模型API生成最终的网页代码。

In [3]:
def download_video(video_url, output_dir="temp_video"):
    """
    使用 yt-dlp 从指定URL下载视频。
    参数:
        video_url (str): 视频的URL。
        output_dir (str): 存放下载视频的临时目录。
    返回:
        str: 视频文件的本地路径，如果失败则返回None。
    """
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir)
    print(f">>> 正在下载视频: {video_url}")
    
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best',
        'outtmpl': os.path.join(output_dir, 'video.%(ext)s'),
        'quiet': True,
        'no_warnings': True,
        'merge_output_format': 'mp4'
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url])
        for file in os.listdir(output_dir):
            if file.startswith("video"):
                print("✅ 视频下载成功。")
                return os.path.join(output_dir, file)
        print("❌ 视频下载失败：未找到已下载的文件。")
        return None
    except Exception as e:
        print(f"❌ 视频下载失败: {e}")
        return None

def extract_frames(video_path, interval_sec=1):
    """
    从视频文件中按指定时间间隔提取图像帧。
    参数:
        video_path (str): 视频文件的本地路径。
        interval_sec (int): 提取帧的时间间隔（秒）。
    返回:
        list: 帧片段的列表，每个片段包含30个Base64编码的图像字符串。
    """
    print(">>> 正在进行视频分帧处理...")
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("❌ 错误：无法打开视频文件。")
        return []
        
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps == 0: fps = 30 

    chunk_frames, chunks, frame_count = [], [], 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break
        
        if frame_count % int(fps * interval_sec) == 0:
            height, width = frame.shape[:2]
            scale = 512 / height
            resized_frame = cv2.resize(frame, (int(width * scale), 512))
            
            _, buffer = cv2.imencode('.jpg', resized_frame, [int(cv2.IMWRITE_JPEG_QUALITY), 60])
            frame_b64 = base64.b64encode(buffer).decode('utf-8')
            chunk_frames.append(frame_b64)
            
            if len(chunk_frames) == 30:
                chunks.append(chunk_frames)
                chunk_frames = []
        frame_count += 1
    
    if chunk_frames:
        chunks.append(chunk_frames)
        
    cap.release()
    print(f"✅ 视频分帧完成，共切分为 {len(chunks)} 个片段。")
    return chunks

def process_chunk_with_retry(client, chunk_index, frames_b64, max_retries=3):
    """
    调用ERNIE API处理单个视频片段，并包含重试逻辑。
    参数:
        client (OpenAI): API客户端实例。
        chunk_index (int): 片段的索引。
        frames_b64 (list): 片段的帧图像Base64字符串列表。
        max_retries (int): 最大重试次数。
    返回:
        tuple: (片段索引, 模型返回的文本描述)。
    """
    prompt = f"这是一段关于网页编程的视频教程片段（每秒一张截图）。请聚焦于屏幕上的代码和最终的网页样式，详细描述这段视频中展示的HTML结构、CSS样式代码或JS交互逻辑。"
    
    content = [{"type": "text", "text": prompt}]
    for f in frames_b64:
        content.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{f}"}})
        
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=MODEL_NAME,
                messages=[{"role": "user", "content": content}],
                temperature=0.1, max_tokens=1024
            )
            return chunk_index, response.choices[0].message.content
        except Exception as e:
            if attempt == max_retries - 1:
                tqdm.write(f"❌ 片段 {chunk_index+1} 在 {max_retries} 次尝试后失败: {e}")
            time.sleep(2)
    return chunk_index, ""

def aggregate_and_generate_webpage(client, summaries):
    """
    聚合所有片段的摘要，并调用API生成最终网页代码。
    参数:
        client (OpenAI): API客户端实例。
        summaries (dict): 所有片段的索引及其文本描述的字典。
    返回:
        str: 生成的完整HTML代码。
    """
    print("\n>>> 正在聚合内容并生成最终代码...")
    full_summary = "\n".join([f"片段{i+1}总结: {s}" for i, s in sorted(summaries.items()) if s])
    
    final_prompt = f"""
    你是一名资深的前端工程师。根据下方提供的、从一个编程视频教程中提取的分段摘要，编写一个完整、独立的HTML文件来复现视频中的最终效果。

    **视频分段摘要:**
    ---
    {full_summary}
    ---

    **指令:**
    1.  代码必须严格基于摘要中提到的HTML结构、CSS样式和功能。
    2.  忽略摘要中与编程无关的描述。
    3.  如果摘要信息不完整，请根据上下文合理补全，形成功能完整的网页。
    4.  所有HTML和CSS（在`<style>`标签内）必须在同一个文件中。
    5.  仅返回纯HTML代码，以`<!DOCTYPE html>`开头，不包含任何解释或markdown标记。
    """
    
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": final_prompt}],
        stream=True, temperature=0.2, top_p=0.8
    )
    
    html_code = ""
    print(">>> 模型正在生成代码:\n")
    for chunk in response:
        if hasattr(chunk.choices[0].delta, "content") and chunk.choices[0].delta.content:
            char = chunk.choices[0].delta.content
            print(char, end="", flush=True)
            html_code += char
    return html_code

### 6. 主程序执行

运行以下单元格将启动完整的自动化流程。该脚本会使用在配置部分指定的 `VIDEO_URL`，并按顺序执行所有定义的函数。

In [6]:
def main():
    """主执行函数，负责调度整个工作流。"""
    if "YOUR_ERNIE_API_KEY" in API_KEY or not API_KEY:
        print("❌ 错误：请返回第4步的配置单元格，正确填写您的 API_KEY。")
        return

    # 初始化API客户端
    client = OpenAI(api_key=API_KEY, base_url=BASE_URL)
    
    # 步骤 1 & 2: 下载和分帧
    video_path = download_video(VIDEO_URL)
    if not video_path: return
    
    chunks = extract_frames(video_path)
    if not chunks: return
    
    # 步骤 3: 并发处理所有片段
    print(f">>> 开始并发处理 {len(chunks)} 个视频片段（并发数: {MAX_CONCURRENT_REQUESTS}）...")
    chunk_summaries = {}
    
    with ThreadPoolExecutor(max_workers=MAX_CONCURRENT_REQUESTS) as executor:
        future_to_chunk_index = {executor.submit(process_chunk_with_retry, client, i, chunk): i for i, chunk in enumerate(chunks)}
        
        for future in tqdm(as_completed(future_to_chunk_index), total=len(chunks), desc="片段处理进度"):
            idx, summary = future.result()
            if summary:
                chunk_summaries[idx] = summary
                
    if not chunk_summaries:
        print("\n❌ 所有片段处理失败，无法生成最终代码。")
        return
        
    # 步骤 4: 聚合内容并生成网页
    html_code = aggregate_and_generate_webpage(client, chunk_summaries)
    
    # 步骤 5: 保存并打开结果
    output_file = "final_result.html"
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(html_code)
    
    print(f"\n\n>>> 工作流执行完毕！网页已保存为: {output_file}")
    abs_path = os.path.abspath(output_file)
    webbrowser.open(f"file://{abs_path}")
    print(f">>> 正在默认浏览器中打开结果页面...")
    
    # 清理临时文件
    shutil.rmtree("temp_video", ignore_errors=True)
    print(">>> 临时文件已清理。")

# --- 运行主程序 ---
main()

>>> 正在下载视频: https://www.bilibili.com/video/BV1xQ4y167xr/
✅ 视频下载成功。                                                  
>>> 正在进行视频分帧处理...
✅ 视频分帧完成，共切分为 10 个片段。
>>> 开始并发处理 10 个视频片段（并发数: 4）...


片段处理进度: 100%|██████████| 10/10 [01:10<00:00,  7.09s/it]



>>> 正在聚合内容并生成最终代码...
>>> 模型正在生成代码:

<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Flip Card Effect</title>
    <style>
        body {
            display: flex;
            justify-content: center;
            align-items: center;
            min-height: 100vh;
            margin: 0;
            background-color: #1db7c2;
            flex-wrap: wrap;
        }

        .rd-container {
            display: flex;
            flex-wrap: wrap;
            justify-content: center;
        }

        .card {
            position: relative;
            width: 100px;
            height: 100px;
            margin: 8px;
            perspective: 1000px;
            transition: transform 0.6s;
        }

        .cover,
        .back {
            position: absolute;
            width: 100%;
            height: 100%;
            display: flex;
            justify-content: center;
          

### 7. 最终效果演示

In [1]:
from IPython.display import Video

# 嵌入并播放本地视频文件
Video("video2code_demo.mp4", width=800)